In [57]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# 1.Built Environment Variables

## 1.1 POI entropy culculaion

In [61]:
import geopandas as gpd
import os
import numpy as np

# Define file paths
fishnet_path = "./data/shp_data/fishnet_300/Fishnet_300m2.shp"
poi_directory = "./data/shp_data/2021.03/"
poi_files = {
    "edu": "深圳市_科研教育.shp",
    "pbs": "深圳市_居委会点.shp",
    "resid": "深圳市_居民小区点.shp",
    "comm": "深圳市_金融服务.shp",
    "shop": "深圳市_购物.shp",
    "work": "深圳市-写字楼.shp",
    "govern": "深圳市_政府机关.shp",
    "med": "深圳市_医疗服务.shp",
    "res": "深圳市_休闲娱乐.shp"
}

# Read fishnet data
fishnet = gpd.read_file(fishnet_path)

# Set CRS of fishnet data to WGS1984
fishnet = fishnet.to_crs(epsg=4326)

# Initialize statistics, add a column for each POI type
for poi_type in poi_files.keys():
    fishnet[poi_type] = 0

# Count the number of POI points in each fishnet grid
for poi_type, poi_file in poi_files.items():
    poi_path = os.path.join(poi_directory, poi_file)
    poi_gdf = gpd.read_file(poi_path)
    poi_gdf = poi_gdf.to_crs(epsg=4326)

    # Check the loaded POI data and CRS conversion
    print(f"Processing {poi_type} ({poi_file})")
    print(f"POI Data CRS: {poi_gdf.crs}")
    print(f"Fishnet Data CRS: {fishnet.crs}")
    print(f"Number of POI points: {len(poi_gdf)}")

    # Spatial join to count POI points within each grid cell
    joined = gpd.sjoin(poi_gdf, fishnet, how="left", predicate="intersects")

    # Check the result of the spatial join
    print(f"Number of joined records: {len(joined)}")

    # Ensure `index_right` exists and count POI points in each grid cell
    if 'index_right' in joined.columns:
        counts = joined.groupby("index_right").size()
        # Update POI counts in the fishnet grid
        fishnet.loc[counts.index, poi_type] += counts
    else:
        print(f"index_right not found in the joined result for file {poi_file}")

# Function to calculate entropy for each grid cell
def calculate_entropy(row, columns):
    total = sum(row[columns])
    if total == 0:
        return 0
    entropy = 0
    for col in columns:
        p_i = row[col] / total
        if p_i > 0:
            entropy -= p_i * np.log(p_i)
    return entropy

# Compute entropy for each grid cell based on POI diversity
poi_columns = list(poi_files.keys())
fishnet['entropy'] = fishnet.apply(lambda row: calculate_entropy(row, poi_columns), axis=1)

# Display the results
df_poi = fishnet[["geometry"] + poi_columns + ["entropy"]]


Processing edu (深圳市_科研教育.shp)
POI Data CRS: EPSG:4326
Fishnet Data CRS: EPSG:4326
Number of POI points: 6244
Number of joined records: 6244
Processing pbs (深圳市_居委会点.shp)
POI Data CRS: EPSG:4326
Fishnet Data CRS: EPSG:4326
Number of POI points: 629
Number of joined records: 629
Processing resid (深圳市_居民小区点.shp)
POI Data CRS: EPSG:4326
Fishnet Data CRS: EPSG:4326
Number of POI points: 7311
Number of joined records: 7311
Processing comm (深圳市_金融服务.shp)
POI Data CRS: EPSG:4326
Fishnet Data CRS: EPSG:4326
Number of POI points: 7072
Number of joined records: 7072
Processing shop (深圳市_购物.shp)
POI Data CRS: EPSG:4326
Fishnet Data CRS: EPSG:4326
Number of POI points: 71040
Number of joined records: 71040
Processing work (深圳市-写字楼.shp)
POI Data CRS: EPSG:4326
Fishnet Data CRS: EPSG:4326
Number of POI points: 1028
Number of joined records: 1028
Processing govern (深圳市_政府机关.shp)
POI Data CRS: EPSG:4326
Fishnet Data CRS: EPSG:4326
Number of POI points: 2214
Number of joined records: 2214
Processing med

## 1.2 bus distance culculation

In [63]:
df_bus = pd.read_excel('./data/shp_data/bus.xls')
df_bus['IN_FID']=df_bus['IN_FID']-1
df_bus = df_bus.rename(columns={'IN_FID':'ID'})
df_poi = df_poi.reset_index().rename(columns={'index': 'ID'})
df = pd.merge(df_bus, df_poi, on='ID', how='inner')
columns_to_drop = [
    'OBJECTID', 'NEAR_FID', 'geometry', 'edu', 'pbs', 'resid', 'comm', 'shop', 'work', 'govern',
       'med', 'res'
]
df = df.drop(columns=columns_to_drop)

## 1.3 NDVI

In [65]:
import geopandas as gpd
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterstats
import numpy as np

# 定义文件路径
fishnet_path = "./data/shp_data/fishnet_300/Fishnet_300m2.shp"
ndvi_tif_path = "./data/raster_data/NDVI.tif"
output_ndvi_reprojected = "./data/raster_data/NDVI_reprojected.tif"

# 读取渔网数据
fishnet = gpd.read_file(fishnet_path)

# 确保渔网数据的CRS为WGS 1984 UTM Zone 50N
fishnet = fishnet.to_crs(epsg=32650)

# 读取NDVI栅格数据并检查CRS
with rasterio.open(ndvi_tif_path) as src:
    src_crs = src.crs
    if src_crs != fishnet.crs:
        print(f"Reprojecting NDVI raster from {src_crs} to {fishnet.crs}")

        # 计算转换参数
        transform, width, height = calculate_default_transform(
            src_crs, fishnet.crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': fishnet.crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        # 重新投影并保存为新的栅格文件
        with rasterio.open(output_ndvi_reprojected, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=fishnet.crs,
                    resampling=Resampling.nearest)

        ndvi_tif_path = output_ndvi_reprojected

# 初始化结果列
fishnet["NDVI_mean"] = 0

# 读取重新投影的NDVI栅格数据
with rasterio.open(ndvi_tif_path) as src:
    affine = src.transform
    array = src.read(1)

    # 计算渔网格中的平均值
    stats = rasterstats.zonal_stats(fishnet, array, affine=affine, stats=['mean'], nodata=src.nodata)

    # 将统计结果存入渔网数据
    fishnet["NDVI_mean"] = [stat['mean'] if stat['mean'] is not None else np.nan for stat in stats]
df = df.merge(fishnet[['ID', 'NDVI_mean']], on='ID', how='left')

Reprojecting NDVI raster from EPSG:32649 to EPSG:32650


In [66]:
df_dependent = pd.read_csv('./data/variables/variables_remained.csv')
df_dependent = pd.merge(df, df_dependent, on='ID')
df_dependent.rename(columns={'NEAR_DIST': 'bus distance', 'entropy': 'POI diversity',
                  'NDVI_mean':'NDVI',}, inplace=True)

In [67]:
df_dependent.to_csv('./data/variables/variables_constant.csv',index = False)